# Assignment 3

## Question 1

In [1]:
import pandas as pd
import numpy as np

In [75]:
energy = pd.read_excel("Datasets/EnergyIndicators.xls", header=None)
energy = energy[18:245].reset_index().rename(columns={2: 'country', 3: 'Energy Supply', 4: 'Energy Supply per Capita', 5: '% Renewable'})
energy.drop({'index', 0, 1}, inplace=True, axis=1)
energy['Energy Supply'] = np.where(energy['Energy Supply']=='...', np.nan, energy['Energy Supply'])
energy['Energy Supply per Capita'] = np.where(energy['Energy Supply per Capita']=='...', np.nan, energy['Energy Supply per Capita'])
energy['Energy Supply'] = energy['Energy Supply'] * 10**6
energy.head(30)

,country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,321000000,10,78.66928
1,Albania,102000000,35,100
2,Algeria,1959000000,51,0.55101
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9000000,121,88.69565
5,Angola,642000000,27,70.90909
6,Anguilla,2000000,136,0
7,Antigua and Barbuda,8000000,84,0
8,Argentina,3378000000,79,24.06452
9,Armenia,143000000,48,28.23606


In [76]:
energy['country'] = energy['country'].replace('\d*', '',regex=True)
energy['country'] = energy['country'].replace('\s*\(.*\)', '', regex=True)
energy[43:]

,country,Energy Supply,Energy Supply per Capita,% Renewable
43,"China, Hong Kong Special Administrative Region",585000000,82,0
44,"China, Macao Special Administrative Region",47000000,83,0
45,Colombia,1513000000,32,63.00284
46,Comoros,5000000,7,11.62791
47,Congo,102000000,23,57.74887
...,...,...,...,...
222,Viet Nam,2554000000,28,45.32152
223,Wallis and Futuna Islands,0,26,0
224,Yemen,344000000,13,0
225,Zambia,400000000,26,99.71467


In [77]:
old_names = ["Republic of Korea", "United States of America",
                 "United Kingdom of Great Britain and Northern Ireland", 
                 "China, Hong Kong Special Administrative Region"]
new_names = ["South Korea", "United States", "United Kingdom", "Hong Kong"]
energy['country'] = energy['country'].replace(old_names,new_names)
energy[43:]

,country,Energy Supply,Energy Supply per Capita,% Renewable
43,Hong Kong,585000000,82,0
44,"China, Macao Special Administrative Region",47000000,83,0
45,Colombia,1513000000,32,63.00284
46,Comoros,5000000,7,11.62791
47,Congo,102000000,23,57.74887
...,...,...,...,...
222,Viet Nam,2554000000,28,45.32152
223,Wallis and Futuna Islands,0,26,0
224,Yemen,344000000,13,0
225,Zambia,400000000,26,99.71467


In [5]:
GDP = pd.read_csv('Datasets/API/API_NY.csv')
GDP.head()

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 67
